# Structured Data -CSV and Excel Parsing

In [ ]:
import pandas as pd
import os


## Create CSV and Excel

In [ ]:
os.makedirs("data/structured", exist_ok=True)

# create data
data = {
    'Product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Webcam'],
    'Category': ['Electronics', 'Accessories', 'Accessories', 'Electronics', 'Electronics'],
    'Price': [999.99, 29.99, 79.99, 299.99, 89.99],
    'Stock': [50, 200, 150, 75, 100],
    'Description': [
        'High-performance laptop with 16GB RAM and 512GB SSD',
        'Wireless optical mouse with ergonomic design',
        'Mechanical keyboard with RGB backlighting',
        '27-inch 4K monitor with HDR support',
        '1080p webcam with noise cancellation'
    ]
}

# Save as CSV
df = pd.DataFrame(data)
df.to_csv('data/structured/products.csv', index=False)

# Save data as Excel with multiple sheets
with pd.ExcelWriter('data/structured/inventory.xlsx') as writer:
    df.to_excel(writer, sheet_name='Products', index=False)
    
    # Add another sheet
    summary_data = {
        'Category': ['Electronics', 'Accessories'],
        'Total_Items': [3, 2],
        'Total_Value': [1389.97, 109.98]
    }
    pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)

## CSV Processing

In [ ]:
from langchain_community.document_loaders import CSVLoader, UnstructuredCSVLoader

### Methods 1 - Each Row Become as Document

In [ ]:
csv_loader = CSVLoader(
    file_path='data/structured/products.csv',
    encoding='utf-8',
    csv_args={
        'delimiter': ',',
        'quotechar': '"',
    }
)

csv_docs = csv_loader.load()
print(f"loaded {len(csv_docs)} data in the CSV")
print("------"*5)
print(f"First doc Content:\n {csv_docs[0].page_content}")
print("------"*5)
print(f"First doc Metadata:\n {csv_docs[0].metadata}")

### Method 2: Custom CSV With Structure and Metadata

In [ ]:
from typing import List
from langchain_core.documents import Document


def process_csv_intelligently(filepath: str) -> List[Document]:
    """Process CSV with intelligent document creation"""
    df = pd.read_csv(filepath)
    documents = []
    
    for idx, row in df.iterrows():
        # Create structured content
        content = f"""Product Information:
        Name: {row['Product']}
        Category: {row['Category']}
        Price: ${row['Price']}
        Stock: {row['Stock']} units
        Description: {row['Description']}"""
        
        # Create document with rich metadata
        doc = Document(
            page_content=content,
            metadata={
                'source': filepath,
                'row_index': idx,
                'product_name': row['Product'],
                'category': row['Category'],
                'price': row['Price'],
                'data_type': 'product_info'
            }
        )
        documents.append(doc)
    return documents

In [ ]:
process_csv_intelligently('data/structured/products.csv')

# 📊 CSV Processing Strategies
print("\n📊 CSV Processing Strategies:")
print("\n1. Row-based (CSVLoader):")
print("  ✅ Simple one-row-one-document")
print("  ✅ Good for record lookups")
print("  ❌ Loses table context")

print("\n2. Intelligent Processing:")
print("  ✅ Preserves relationships")
print("  ✅ Creates summaries")
print("  ✅ Rich metadata")
print("  ✅ Better for Q&A")

## Excel Processing

### Pandas for Full Control

In [ ]:

def process_excel_with_pandas(filepath: str) -> List[Document]:
    """Process Excel with sheet awareness"""
    documents = []
    
    # Read all sheets
    excel_file = pd.ExcelFile(filepath)
    
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel(filepath, sheet_name=sheet_name)
        
        # Create document for each sheet
        sheet_content = f"Sheet: {sheet_name}\n"
        sheet_content += f"Columns: {', '.join(df.columns)}\n"
        sheet_content += f"Rows: {len(df)}\n\n"
        sheet_content += df.to_string(index=False)
        
        doc = Document(
            page_content=sheet_content,
            metadata={
                'source': filepath,
                'sheet_name': sheet_name,
                'num_rows': len(df),
                'num_columns': len(df.columns),
                'data_type': 'excel_sheet'
            }
        )
        documents.append(doc)
    
    return documents

In [ ]:
excel_docs = process_excel_with_pandas('data/structured/inventory.xlsx')
print(f"Processed {len(excel_docs)} sheets\n--------")
excel_docs

### Unstructured Excel Loader

In [36]:
from langchain_community.document_loaders import UnstructuredExcelLoader
try:
    excel_loader = UnstructuredExcelLoader(
        'data/structured/inventory.xlsx',
        mode="elements"
    )
    unstructured_docs = excel_loader.load()
    print("  ✅ Handles complex Excel features")
    print("  ✅ Preserves formatting info")
except Exception as e:
    print(f"error :\n {e}")


print(f"length of docs {len(unstructured_docs)}")
for i, doc in enumerate(unstructured_docs):
    print(f"doc {i} :\n {doc}")

  ✅ Handles complex Excel features
  ✅ Preserves formatting info
length of docs 2
doc 0 :
 page_content='Product Category Price Stock Description Laptop Electronics 999.99 50 High-performance laptop with 16GB RAM and 512GB SSD Mouse Accessories 29.99 200 Wireless optical mouse with ergonomic design Keyboard Accessories 79.99 150 Mechanical keyboard with RGB backlighting Monitor Electronics 299.99 75 27-inch 4K monitor with HDR support Webcam Electronics 89.99 100 1080p webcam with noise cancellation' metadata={'source': 'data/structured/inventory.xlsx', 'file_directory': 'data/structured', 'filename': 'inventory.xlsx', 'last_modified': '2025-08-24T20:10:18', 'page_name': 'Products', 'page_number': 1, 'text_as_html': '<table><tr><td>Product</td><td>Category</td><td>Price</td><td>Stock</td><td>Description</td></tr><tr><td>Laptop</td><td>Electronics</td><td>999.99</td><td>50</td><td>High-performance laptop with 16GB RAM and 512GB SSD</td></tr><tr><td>Mouse</td><td>Accessories</td><td>29.9